---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [3]:
import pandas as pd
from datetime import datetime
date_input = '20190707'
datetimeobject = datetime.strptime(date_input,'%Y%m%d')
print(datetimeobject)
doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head(10)

2019-07-07 00:00:00


0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
5                    .Per 7/06/79 Movement D/O note:\n
6    4, 5/18/78 Patient's thoughts about current su...
7    10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                         3/7/86 SOS-10 Total Score:\n
9             (4/10/71)Score-1Audit C Score Current:\n
dtype: object

In [1]:
def date_sorter():
    ### import relevant libraries
    from datetime import datetime #to convert strings into dates
    import re #to use regex
    
    def convert_date_logic(date):
        # check length of date string
        date_length = len(date)
    
        # initialise logic variables:
        conv_date = None 
        fwd_slash = 0
        hyphen = 0
        letters = False
        letters_start = False
        month_short = False
        month_short_start = False
        comma = False
        year_2digits = False
        
        # check how many forward slash '/' date contains:
        if date.count('/') > 1:
            fwd_slash = 2
        
        if date.count('/') == 1:
            fwd_slash = 1
        
        # check how many hyphens '-' date contains:
        if date.count('-') > 1:
            hyphen = 2
        
        if date.count('-') == 1:
            hyphen = 1

        # check if contains letters:
        if len(re.findall(r'[A-Za-z]',date))>0:
            letters = True

        # check if contains letters at the beginning:
        if len(re.findall(r'^[A-Za-z]',date))>0:
            letters_start = True

        # check if contains month abbreviation:
        if len(re.findall(r'Jan\b|Feb\b|Mar\b|Apr\b|May\b|Jun\b|Jul\b|Aug\b|Sep\b|Oct\b|Nov\b|Dec\b',date)) > 0: ## possibly remove May
            month_short = True

        # check if contains month abbreviation at the beginning:
        if len(re.findall(r'^Jan\b|^Feb\b|^Mar\b|^Apr\b|^May\b|^Jun\b|^Jul\b|^Aug\b|^Sep\b|^Oct\b|^Nov\b|^Dec\b',date)) > 0: ## possibly remove May
            month_short_start = True

        # check if contains comma:
        if ',' in date:
            comma = True

        # check if year is 2 digits:
        if len(re.findall(r'\d{4}$',date)) == 0:
            year_2digits = True

        ### correct typos (Janaury and Decemeber):
        date = date.replace("Janaury", "January")
        date = date.replace("Decemeber", "December")

        ### dates conversion: 
        if fwd_slash == 2:
            if date_length > 8:
                # convert dates with format [MM/DD/YYYY]
                conv_date = datetime.strptime(date, "%m/%d/%Y")
            else:
                # convert dates with format [MM/DD/YY]
                conv_date = datetime.strptime(date, "%m/%d/%y")
        
        if fwd_slash == 1:
            if re.match(r"\d{1,2}/\d{2}\b", date):
                conv_date = datetime.strptime(date, "%m/%y")
            elif re.match(r"\d{1,2}/\d{4}\b", date):
                conv_date = datetime.strptime(date, "%m/%Y")
        
        if hyphen == 2:
            if date_length > 8:
                # convert dates with format [MM-DD-YYYY]
                conv_date = datetime.strptime(date, "%m-%d-%Y")
            else:
                # convert dates with format [MM-DD-YY]
                conv_date = datetime.strptime(date, "%m-%d-%y")
        
        if hyphen == 1:
            if re.match(r"\d{1,2}-\d{2}\b", date):
                conv_date = datetime.strptime(date, "%m-%y")
            elif re.match(r"\d{1,2}-\d{4}\b", date):
                conv_date = datetime.strptime(date, "%m-%Y")

        if date_length == 4:
            # convert dates with format [YYYY]
            conv_date =  datetime.strptime(date, "%Y")

        if letters and (not letters_start) and (not month_short) and (not comma) and (not year_2digits):
            # convert dates with format [DD Month YYYY]
            conv_date =  datetime.strptime(date, "%d %B %Y")
        
        if letters_start and (not month_short) and (not comma) and (not year_2digits):
            if re.match(r"(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]+ \d{2} \d{4}", date):
                conv_date =  datetime.strptime(date, "%B %d %Y")
            else:
                # convert dates with format [Month YYYY]
                conv_date =  datetime.strptime(date, "%B %Y")
                
        if month_short and (not month_short_start) and (not comma) and (not year_2digits):
            # convert dates with format [DD Month_abbreviation YYYY]
            conv_date =  datetime.strptime(date, "%d %b %Y")
        
        if month_short_start and (not comma) and (not year_2digits):
            if re.match(r"(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec) \d{2} \d{4}", date):
                conv_date =  datetime.strptime(date, "%b %d %Y")
            else:
                # convert dates with format [Month_abbreviation YYYY]
                conv_date =  datetime.strptime(date, "%b %Y")

        if letters_start and (not month_short) and comma and (not year_2digits):
            if re.match(r"^[A-Za-z]+,", date):
                conv_date =  datetime.strptime(date.replace(',', '').replace('.', ''), "%B %Y")
            else:
                # convert dates with format [Month DD, YYYY]. July, 1990
                conv_date =  datetime.strptime(date.replace(',', '').replace('.', ''), "%B %d %Y")

        if month_short_start and comma and (not year_2digits):
            if re.match(r"^[A-Za-z]+,", date):
                conv_date =  datetime.strptime(date.replace(',', '').replace('.', ''), "%b %Y")
            # convert dates with format [Month_abbreviation DD, YYYY]
            else:
                conv_date =  datetime.strptime(date.replace(',', '').replace('.', ''), "%b %d %Y")

        if letters_start and (not month_short) and (not comma) and year_2digits:
            # convert dates with format [Month YY]
            conv_date =  datetime.strptime(date, "%B %y")

        if month_short_start and (not comma) and year_2digits:
            # convert dates with format [Month_abbreviation YY]
            conv_date =  datetime.strptime(date, "%b %y")
        
        if re.match(r"\d{2} (?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec) \d{4}", date):
            conv_date =  datetime.strptime(date, "%d %b %Y")
        
        if re.match(r"\d{2} (?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]+ \d{4}", date):
            conv_date =  datetime.strptime(date, "%d %B %Y")

        if conv_date:
            if conv_date.year < 2022: # check the year is reasonable
                return conv_date

    
    patterns = [r'\d{1,2}[/-]\d{1,2}[/-]\d{2}\b', ## examples: [03/05/21], [4-13-82]
            r'\d{1,2}[/-]\d{1,2}[/-]\d{4}\b', ## examples: [03/05/2021], [4-13-1982]
            r'\d{1,2}[/-]\d{4}\b', ## examples: [6/1998]
            r'\d{1,2} (?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*,? \d{2,4}[^,.\n ]', ## examples: [21 October 2020], [20 Oct 2020], [19 April, 2010]
            r'(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]* \d{4}', ## examples: [Jan 24], [April 1988]
            r'(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*, \d{4}', ## examples: [May, 2019]
            r'(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*.? \d{1,2}, \d{4}', ## examples: [April 11, 1990]
            r'[1-2][0-9][0-9][0-9]\b', # examples: [2010]
            r'(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]* \d{2} \d{2,4}', ## examples: [Jan 24], [April 1988]
            r'^$' #map no match
            ]
    
    pattern = "|".join(patterns)

    list1 = df.str.findall("|".join(patterns)).map(lambda x: x[0])

    converted_date_list = [convert_date_logic(x) for x in list1]
    
    final_order = sorted(range(len(converted_date_list)), key=lambda k: (converted_date_list[k] is None, converted_date_list[k]))
 
    final_order_pd = pd.Series(final_order)
 
    return final_order_pd # Your answer here



In [4]:
print (date_sorter())

0        9
1       84
2        2
3       53
4       28
5      474
6      153
7       13
8      129
9       98
10     111
11     225
12      31
13     171
14     191
15     486
16     335
17     415
18      36
19     323
20     405
21     422
22     375
23     380
24     345
25      57
26     481
27     436
28     104
29     299
      ... 
470    208
471    243
472    139
473    320
474    383
475    244
476    286
477    480
478    431
479    279
480    198
481    381
482    463
483    366
484    255
485    439
486    401
487    475
488    257
489    152
490    235
491    464
492    253
493    231
494    427
495    141
496    186
497    161
498    413
499    271
Length: 500, dtype: int64
